In [1]:
# Required imports
try:
    import polars as pl
except ImportError:
    !pip install polars
    import polars as pl
    
try:
    from pyproj import Transformer
except ImportError:
    !pip install pyproj
    from pyproj import Transformer
    
try:
    import folium
except ImportError:
    !pip install folium
    import folium

import requests
from datetime import datetime

In [2]:
wtype_mapping = {
    1: "Thunderstorm Warning",
    2: "Flood Warning",
    3: "Snow Warning",
    4: "Wind Warning",
    5: "Heat Warning",
    6: "Rain Warning",
    7: "Severe Weather Alert"
}

url = "https://warnungen.zamg.at/wsapp/api/getWarnstatus"

transformer = Transformer.from_crs("EPSG:31287", "EPSG:4326", always_xy = True)

try:
    response = requests.get(url)
    response.raise_for_status() 
    data = response.json()
except Exception as e:
    print("Error fetching data:", e)
    data = None

rows = []

if data:
    for feature in data.get("features", []):
        # Extract properties and geometry
        properties = feature.get("properties", {})
        geometry = feature.get("geometry", {})
        geom_coords = geometry.get("coordinates", [])

        warnid = properties.get("warnid")
        wtype_code = properties.get("wtype")
        wtype = wtype_mapping.get(wtype_code)
        wlevel = properties.get("wlevel")
        
        # Convert time
        start = datetime.fromtimestamp(int(properties.get("start")))
        end = datetime.fromtimestamp(int(properties.get("end")))
        
        gemeinden = properties.get("gemeinden", [])

        # Extract coordinate pairs and convert them
        coordinate_pairs = []
        for polygon in geom_coords:  
            for ring in polygon:  
                for point in ring: 
                    # Convert coordinates from EPSG:31287 to WGS84
                    x, y = point
                    lon, lat = transformer.transform(x, y)
                    coordinate_pairs.append((lon, lat))
  
        while len(coordinate_pairs) < len(gemeinden):
            coordinate_pairs.append((None, None))  

        # Create a row for each gemeinde with one coordinate pair
        for gemeinde, coord in zip(gemeinden, coordinate_pairs):
            rows.append({
                "lon": coord[0],
                "lat": coord[1],
                "warnid": warnid,
                "wtype": wtype,
                "wlevel": wlevel,
                "start": start,
                "end": end,
                "gemeinde": gemeinde
            })
    
    # Create DataFrame and save as Parquet
    df = pl.DataFrame(rows)
    df.write_parquet("./Data/Weather/warnungen.parquet")
    print("Data has been successfully written to Parquet.")
else:
    print("No data available from the API.")

Data has been successfully written to Parquet.


In [7]:
df = pl.read_parquet("./Data/Weather/warnungen.parquet")
df

lon,lat,warnid,wtype,wlevel,start,end,gemeinde
f64,f64,str,str,i64,datetime[μs],datetime[μs],str
13.827378,48.174517,"""w20c202502200v11""","""Severe Weather Alert""",1,2025-02-19 23:00:00,2025-02-20 22:59:00,"""10101"""
13.820114,48.174421,"""w20c202502200v11""","""Severe Weather Alert""",1,2025-02-19 23:00:00,2025-02-20 22:59:00,"""10201"""
13.816429,48.156957,"""w20c202502200v11""","""Severe Weather Alert""",1,2025-02-19 23:00:00,2025-02-20 22:59:00,"""10301"""
13.810287,48.157216,"""w20c202502200v11""","""Severe Weather Alert""",1,2025-02-19 23:00:00,2025-02-20 22:59:00,"""10302"""
13.799889,48.150655,"""w20c202502200v11""","""Severe Weather Alert""",1,2025-02-19 23:00:00,2025-02-20 22:59:00,"""10303"""
…,…,…,…,…,…,…,…
15.183303,47.756722,"""w20c202502200v11""","""Severe Weather Alert""",1,2025-02-19 23:00:00,2025-02-20 22:59:00,"""91901"""
15.168545,47.753727,"""w20c202502200v11""","""Severe Weather Alert""",1,2025-02-19 23:00:00,2025-02-20 22:59:00,"""92001"""
15.158966,47.758424,"""w20c202502200v11""","""Severe Weather Alert""",1,2025-02-19 23:00:00,2025-02-20 22:59:00,"""92101"""
